In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)

In [ ]:
df_cluster_id = pd.read_csv('Final_Cluster_300k.csv')

In [ ]:
df_cluster = pd.read_csv('Cluster_Tweet_Final.csv')
df_cluster = df_cluster.dropna(subset=['Cluster_ID'])
df_cluster = df_cluster.dropna(subset = 'User_ID')
df_cluster.User_ID = df_cluster.User_ID.astype(int)
df_cluster = df_cluster.dropna(subset=['Cluster_ID'])

In [ ]:
df_cluster = df_cluster.dropna(subset = 'User_ID')

In [ ]:
df_cluster.User_ID = df_cluster.User_ID.astype(int)

In [ ]:
df_cluster = df_cluster.dropna(subset=['Cluster_ID'])

In [ ]:
df_cluster['mainbhichowkidar'] = df_cluster["Tweet_Text"].str.contains("mainbhichowkidar", case=False, na=False).astype(int)

In [ ]:
df_cluster['movement'] = df_cluster["Tweet_Text"].str.contains("movement", case=False, na=False).astype(int)

In [ ]:

df_cluster['BharatJodoYatra'] = df_cluster["Tweet_Text"].str.contains("BharatJodoYatra", case=False, na=False).astype(int)

In [ ]:
df_cluster.loc[df_cluster.mainbhichowkidar != 0].groupby('Cluster_ID').count()

In [ ]:
df_cluster.loc[df_cluster.Cluster_ID == 3782.0].Tweet_Text

In [ ]:
df_cluster.loc[df_cluster.movement != 0][['Cluster_ID', 'Tweet_Text']].sample(10)

In [ ]:
df_cluster.loc[df_cluster.BharatJodoYatra != 0].groupby('Cluster_ID').count().reset_index().sort_values('Tweet_ID').iloc[-10:]

In [ ]:
df_cluster.loc[df_cluster.Cluster_ID == 38672.0][['Cluster_ID', 'Tweet_Text']]

In [ ]:
df_cluster['DT'] = pd.to_datetime(df_cluster['Tweet_Time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_cluster_count = df_cluster.groupby('Cluster_ID').resample('7D', on='DT', origin= 'start').count()

In [ ]:
df_cl_count = df_cluster_count[['Tweet_ID']].reset_index()

In [ ]:
df_cl_count.loc[df_cl_count.Tweet_ID > 0].iloc[0:20]

In [ ]:
df_cluster.groupby('User_ID').count()

In [ ]:
df_cluster

In [ ]:
df_cluster.loc[df_cluster.movement != 0][['Cluster_ID', 'Tweet_Text']].sample(20)

In [ ]:
df_cluster['PMCARES'] = df_cluster["Tweet_Text"].str.contains("PMCARES", case=False, na=False).astype(int)
df_cluster['movement_hin'] =  df_cluster["Tweet_Text"].str.contains("आंदोलन", case=False, na=False).astype(int)

In [ ]:
df_cluster.loc[df_cluster.PMCARES != 0][['Cluster_ID', 'Tweet_Text']]

In [ ]:
df_cluster.loc[df_cluster.movement_hin != 0][['Cluster_ID', 'Tweet_Text']].sample(20)

In [ ]:
df_cluster.columns

In [ ]:
doc2 = df_cluster.loc[df_cluster.Tweet_Lang == 'en'].drop_duplicates('Cluster_ID').Tweet_Text.tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dstefa/roberta-base_topic_classification_nyt_news")
model = AutoModelForSequenceClassification.from_pretrained("dstefa/roberta-base_topic_classification_nyt_news")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

text = "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his innocence and vowing."
pipe(text)


In [ ]:
text = df_cluster.loc[df_cluster.Tweet_Lang == 'en'].drop_duplicates('Cluster_ID').Tweet_Text[6]
pipe(text)

In [ ]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

    
MODEL = f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

text = "The whole country wants to know whether this is a farmers' movement, right? Because slogans of 'Kisan Zindabad' should be raised in the farmers' meeting, instead of them slogans of 'Allah Hu Akbar' are being raised. Since when did the slogan of 'Allah Hu Akbar' start serving the farmers' welfare? Is this the politics of Tikait and Salim Yadav? understand"
tokens = tokenizer(text, return_tensors='pt')
output = model(**tokens)

scores = output[0][0].detach().numpy()
scores = expit(scores)
predictions = (scores >= 0.5) * 1



for i in range(len(predictions)):
  if predictions[i]:
    print(class_mapping[i])

In [ ]:
pip install bertopic

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
 

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(doc2[0:200])

In [ ]:
topic_model.get_topic_info()

In [ ]:
from bertopic.representation import KeyBERTInspired

representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

In [ ]:
topics, probs = topic_model.fit_transform(doc2[0:5000])

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.topic_labels_

In [ ]:
from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from sklearn.datasets import fetch_20newsgroups


main_representation = KeyBERTInspired()

aspect_model1 = PartOfSpeech("en_core_web_sm")
aspect_model2 = [KeyBERTInspired(top_n_words=30), MaximalMarginalRelevance(diversity=.5)]

representation_model = {
   "Main": main_representation,
   "Aspect1":  aspect_model1,
   "Aspect2":  aspect_model2 
}
topic_model2 = BERTopic(representation_model=representation_model).fit(doc2)